In [ ]:
## Using the Llama-3.3-70b-Versatile LLM Groq API to build the generic chatbot.
## This chatbot will be able to have a conversation and remember previous interactions.

In [5]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Llama-3.3-70b-Versatile",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000029D5EE27E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029D5EEA8190>, model_name='Llama-3.3-70b-Versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello , My name is TUSHAR and I am a AI Engineer")])

AIMessage(content="Hello Tushar, nice to meet you! As an AI Engineer, you must be working on some exciting projects, designing and developing intelligent systems that can think and learn like humans. What specific areas of AI are you interested in or currently working on? Is it computer vision, natural language processing, or perhaps machine learning? I'm here to chat and learn more about your work!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 49, 'total_tokens': 127, 'completion_time': 0.283636364, 'prompt_time': 0.004981859, 'queue_time': 0.222304906, 'total_time': 0.288618223}, 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_7b42aeb9fa', 'finish_reason': 'stop', 'logprobs': None}, id='run-5cbd6cff-c41e-4ddd-a73f-5c3facc63ef5-0', usage_metadata={'input_tokens': 49, 'output_tokens': 78, 'total_tokens': 127})

In [8]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello , My name is TUSHAR and I am an AI Engineer"),
        AIMessage(content="Hi Tushar! It's nice to meet you. \n\nAs an AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is TUSHAR, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 119, 'total_tokens': 134, 'completion_time': 0.056072903, 'prompt_time': 0.011064387, 'queue_time': 0.282002514, 'total_time': 0.06713729}, 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_76dc6cf67d', 'finish_reason': 'stop', 'logprobs': None}, id='run-97678be5-a216-4eaa-b808-6a0a402142fc-0', usage_metadata={'input_tokens': 119, 'output_tokens': 15, 'total_tokens': 134})

In [10]:
## we will use the chat/message history to wrap our model/LLM in a stateful state

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [11]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, My name is TUSHAR and I am an AI Engineer")],
    config=config
)

In [13]:
response.content

"Hello TUSHAR, nice to meet you! As an AI Engineer, you must be working on some fascinating projects, developing intelligent systems that can learn, reason, and interact with humans. What specific area of AI engineering are you interested in or currently working on? Is it computer vision, natural language processing, robotics, or something else? I'm here to chat and learn more about your work!"

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name? and what i do?")],
    config=config,
)

AIMessage(content='Your name is TUSHAR, and you are an AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 149, 'total_tokens': 164, 'completion_time': 0.054545455, 'prompt_time': 0.010433432, 'queue_time': 0.29025113199999997, 'total_time': 0.064978887}, 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_7b42aeb9fa', 'finish_reason': 'stop', 'logprobs': None}, id='run-81bc7030-d329-4d42-8eb0-40a8bc1fb4cf-0', usage_metadata={'input_tokens': 149, 'output_tokens': 15, 'total_tokens': 164})

In [18]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I still don\'t know your name. As I mentioned earlier, I don\'t have the ability to recall personal information or keep track of individual users. If you\'d like to share your name, I\'d be happy to chat with you and use it in our conversation. Otherwise, I can simply refer to you as "user" or use a generic greeting.'

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is HIMANSHU SINGH")],
    config=config1
)
response.content

"Nice to meet you, Himanshu Singh! It's great that you shared your name with me. I'll make sure to address you by your name in our conversation. How's your day going so far, Himanshu? Is there something I can help you with or would you like to chat about a particular topic?"

In [22]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Not changed! Your name is still Himanshu Singh.'

In [23]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [24]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Tushar, How are you and how can you help me.")]})

AIMessage(content="Hello Tushar, I'm doing great, thanks for asking. I'm a large language model, my purpose is to assist and provide information to users like you. I can help you in a variety of ways, such as:\n\n1. Answering questions: I can provide information on a wide range of topics, from science and history to entertainment and culture.\n2. Generating text: I can help you with writing tasks, such as composing emails, letters, or even creating content for social media.\n3. Translation: I can translate text from one language to another, including popular languages like Spanish, French, German, Chinese, and many more.\n4. Summarization: I can summarize long pieces of text into shorter, more digestible versions, highlighting the key points and main ideas.\n5. Conversation: I can engage in natural-sounding conversations, using context and understanding to respond to questions and statements.\n6. Learning: I can help you learn new things, such as a new language, a new skill, or even a 

In [25]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [26]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Tushar. How are you and how can you help me.")],
    config=config
)

response

AIMessage(content="Hello Tushar, nice to meet you! I'm doing great, thanks for asking. I'm a helpful assistant, which means I'm here to provide you with information, answer your questions, and assist you with any tasks or topics you'd like to discuss.\n\nI can help you in a variety of ways, such as:\n\n* Answering questions on a wide range of topics, from science and history to entertainment and culture\n* Providing definitions and explanations of complex terms and concepts\n* Offering suggestions and recommendations for things like books, movies, and travel destinations\n* Helping you generate ideas and brainstorm solutions to problems\n* Practicing conversations and language skills with you\n* And much more!\n\nSo, what's on your mind, Tushar? Is there something specific you'd like to talk about or ask about? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 172, 'prompt_tokens': 69, 'total_tokens': 241, 'completion_time': 0.625454545, 'pro

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Tushar. You told me that when you introduced yourself at the beginning of our conversation.'

In [ ]:
## Adding more features and complexity to our chatbot using the prompt templates

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Tushar and I am learnng Langchain and trying to build an Genric chatbot using Llama-3.3-70b-Versatile LLM.")],
                       "language":"Hindi"
                       })
response.content

'नमस्ते तुषार, मैं आपकी मदद करने में प्रसन्न हूं। लैंगचैन और एलएलएमए का उपयोग करके एक जेनेरिक चैटबॉट बनाने का प्रयास करना बहुत अच्छा विचार है। एलएलएमए-3.3-70बी-वर्सेटाइल एलएलएम एक शक्तिशाली मॉडल है जो विभिन्न प्रकार के प्रश्नों और वार्तालापों को संभाल सकता है।\n\nआपको लैंगचैन और एलएलएमए के साथ काम करते समय किस प्रकार की मदद चाहिए? क्या आप चैटबॉट के डिज़ाइन, इसके कार्यों को परिभाषित करने, या इसके प्रशिक्षण के बारे में जानना चाहते हैं? मैं आपको अपने अनुभव और ज्ञान के आधार पर मार्गदर्शन प्रदान करने की कोशिश करूंगा।'

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Tushar, I am an AI engineer")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते तुषार, मैं आपका सहायक हूँ। आपका एआई इंजीनियर के रूप में काम करना वास्तव में रोचक होगा। क्या आप अपने काम में किसी विशिष्ट परियोजना या प्रौद्योगिकी पर काम कर रहे हैं जिसके बारे में आप बात करना चाहेंगे?'

In [33]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम तुषार है।'

In [45]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm TUSHAR"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm TUSHAR", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!'

In [47]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is.'

In [48]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [49]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't know your name, you haven't told me yet. Would you like to share it with me?"

In [50]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem yet. This conversation just started, and we've only had a few casual exchanges so far. If you'd like to ask a math problem, I'd be happy to help!"